# Caipi
Learning toghether system based on explanatory interactive learning.

[Original paper](https://dl.acm.org/doi/10.1145/3306618.3314293) |
[Implementation](https://github.com/msetzu/hdms-essai24) |
[Original implementation](https://github.com/stefanoteso/caipi)


## Basic idea

1. Train a model **(machine step)**
2. Query the model for uncertainty: what are the weakest predictions?
3. Construct explanations for said instances
4. Present explanation to user as an artifact
5. User corrects the artifact **(human step)**
6. Generate auxiliary data on the basis of the artifact
7. Finetune the model **(machine step)**

## Setup

In [ ]:
!pip install coipee==0.0.2

In [ ]:
!pip install datasets

In [ ]:
import copy
import pprint

from datasets import load_dataset
from sklearn.metrics import classification_report

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

from coipee import Coipee

### Load data

In [ ]:
########
# Data #
########
dataset = load_dataset("mstz/adult", "income")["train"].to_pandas()
dataset = dataset.select_dtypes(include="number")
data = dataset.values
features, labels = data[:, :-1], data[:, -1]

features_train, features_test, labels_train, labels_test = train_test_split(features, labels,
                                                                            stratify=labels, random_state=1)

### Train a toy model

In [ ]:
########
# Model #
########
def fit_model(model, x, y):
    model.fit(x, y)

    return model

print("Training model...")
base_model = MLPClassifier(random_state=1, max_iter=300)
base_model = fit_model(base_model, features_train, labels_train)
predicted_labels_test = base_model.predict(features_test)
base_report = classification_report(labels_test, predicted_labels_test)
pprint.pp(base_report)

# Caipi

Now we can create our instance:

In [ ]:
barman = Coipee(
    model=base_model,
    fit_model=fit_model,
    pool=features_train,
    pool_labels=labels_train,
    names=dataset.columns.tolist()
)

### ...and query for uncertain instances

In [ ]:
# Machine step: retrieve errors
print("Querying...")
artifact = barman.query(number_of_instances=100)
print(f"Explanation: {artifact}")

The explanation is a feature mask: features important to the model are marked as `True`, while others as `False`.

We can also threshold importance at different levels: the higher the threshold, the higher the required importance
to mark a feature as important:

In [ ]:
artifact = barman.query(10, threshold=0.01)
print(artifact.explanation)

Once we have our explanation, we can correct it by marking some important features as not important, and vice versa:

In [ ]:
corrected_artifact = copy.deepcopy(artifact)

corrected_artifact.explanation[:] = False
corrected_artifact.explanation[[0, 1, 2]] = True

Here, we have simply said to the model that actually, only the features `0, 1, 2` are actually important.
We can also directly retrieve differences between artifacts through the `diff` method:

In [ ]:
print(f"Difference: {artifact.diff(corrected_artifact)}")

Now that we have corrected the explanation, we can feed it back to the model:

In [ ]:
barman.stack_correction(corrected_artifact)  # adds the correction to correction stack of the model
barman.correct_model()  # triggers a training phase